● Create a model that predicts the dropoff_distance of an order, given some features about the
order. A model like this could then be fed into the logistics rider-scheduling algorithms.<br/>
● Propose some front-end features that might help reduce dropoff_distance.
# To Do
1. EDA
2. Missing value and outlier treatment
3. Feature selection
4. Feature Engg
5. Model
6. Error analysis
6. Model understanding
7. Next Steps

In [9]:
import json
import pandas as pd
from pandas_profiling import ProfileReport

In [18]:
# data = json.load(open("../../location_task/location_task_no_nulls.json", 'rb'))
# data_df = pd.DataFrame(data)

data_df = pd.read_csv('../data/processed_data/data_sample.csv')

# Created sample of 10k to reduce experimentation times
# data_df.sample(10000).to_csv('data_sample.csv', index=False)

In [37]:
data_df.created_timestamp_local = pd.to_datetime(data_df.created_timestamp_local, infer_datetime_format=True)

In [38]:
data_df.head()

,order_id,order_value,order_items_count,customer_id,created_timestamp_local,has_instruction,logistics_dropoff_distance,delivery_postal_code,delivery_geohash_precision8,has_phone_number,has_call_instruction,has_leave_instruction,has_lift_instruction,has_lobby_instruction,has_gate_instruction,has_knock_instruction,has_bell_instruction
0,y7ge-p8qa,26.6,3,sgrrf0nl,2021-03-06 22:34:32,True,35.674674,200804,w21z7qhg,False,False,False,False,False,False,False,False
1,n3qu-jf22,38.7,1,sg4q5gu1,2021-03-05 09:29:50,True,18.494004,479245,w21zsgxh,True,False,False,False,False,False,False,False
2,a0ae-x3pg,13.0,1,sgfjwiep,2021-03-20 22:13:49,True,7.272063,NaN,w21z66rg,False,False,False,False,False,False,True,False
3,x1fi-03bv,6.9,1,w9qee4jq,2021-03-09 13:55:53,True,48.315706,534208,w21zu3fx,False,False,False,False,False,False,False,False
4,i4eh-xpfh,10.2,3,sgjgw3du,2021-03-23 19:10:36,True,66.286650,560538,w21zg4rw,False,False,False,True,False,True,False,False


In [6]:
data_df.shape

(2593799, 17)

In [21]:
profile = ProfileReport(data_df, title="Delivery Data Report", explorative=True)

# EDA 
Observations
1. High cardinality for order_id, customer_id, timestamp, postal code, geohash. So these features need to be treated before use or should be ignored for modeling activity depending on what they bring to the table. 
    1. Geohash can be converted into 8 features of 1 bit each
    1. Time stamp can be converted into hour, day of the week, day of the month
    1. customer_id can be changed to # of historical orders made
1. High correlation between has_* fields. So, some dimentionality reduction might be helpful. 

In [22]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

# Feature Engineering
1. Time feature engg
1. Geohash feature engg

In [43]:
data_df['created_timestamp_local_dayofweek'] = data_df.created_timestamp_local.dt.dayofweek
data_df['created_timestamp_local_day'] = data_df.created_timestamp_local.dt.day
data_df['created_timestamp_local_hour'] = data_df.created_timestamp_local.dt.hour

In [72]:
for idx in range(1, 9):
    data_df[f'delivery_geohash_precision8_{idx}'] = data_df.delivery_geohash_precision8.str.split('').str[idx]